<img src="https://gitlab.version.fz-juelich.de/grosch1/pvlink_demo_for_jupytercon/-/raw/JupyterCon2020/img/logo.png"
     style="float:right; width:25%; height:25%; margin-top:25px;">

# Jupyter for interactive in-situ visualization with ParaView/Catalyst

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.version.fz-juelich.de%2Fgrosch1%2Fpvlink_demo_for_jupytercon/JupyterCon2020)  
Try out the *pvlink* extension and the example notebooks in binder.

# Catalyst example

In [1]:
import os
import ipywidgets as widgets

from paraview import simple
from pvlink import RemoteRenderer
from pvlink.utility import SetRecommendedRenderSettings, ResetCamera

## Start a pvserver

In [2]:
%%bash --bg --proc server_process
export OMP_NUM_THREADS=1
pvserver --server-port=11223 > ${JUPYTER_LOG_DIR}/pvserver.log 2>&1

## Start the Catalyst enabled simulation

In [3]:
%%bash --bg --proc catalyst_process
export OMP_NUM_THREADS=1
cd CatalystEnabledSimulation
pvpython ./fedriver.py ./cpscript.py > ${JUPYTER_LOG_DIR}/simulation.log 2>&1

## Setup the RemoteRenderer

In [4]:
jupyter_url = 'jupyter-jsc.fz-juelich.de' + os.environ.get('JUPYTERHUB_SERVICE_PREFIX', '/')
renderer = RemoteRenderer(pvserverHost='localhost', pvserverPort = 11223, 
                          baseURL=jupyter_url, 
                          useJupyterServerProxyHttps=True, disableExternalPort=True)

In [5]:
# Create a view
view1 = simple.FindViewOrCreate('view1', 'RenderView')
SetRecommendedRenderSettings(view1)
view1.Background = [0, 0, 0]
view1.ShowAnnotation = 1
renderer.viewID = view1.GetGlobalIDAsString()

## Establish a Catalyst connection

> Currently, this uses a custom python library patched into ParaView to enable establishing Catayst connections with python without a GUI interface.  
> Starting from ParaView 5.9, a Catalyst python library capable of such a thing should be included by default.

In [6]:
catalyst = simple.CatalystConnection()

In [7]:
# open port for catalyst connection
catalyst.Start()
catalyst.AddUpdateFunction(renderer.update_render)
catalyst.BlockTillConnected();

In [8]:
# wait till simulation connected
catalyst.BlockTillConnected()
# extract data from simulation
# supplying a source name, that can be used to find the ParaView source.
# In case of different named input, or multiple input ports, alows to choose the desired input, that should be extracted
extract = catalyst.Extract('extract')
# block till there is an update for the simulation data
catalyst.BlockTillNextUpdate()
# display simulation data
simple.SetActiveSource(extract)
extractDisplay = simple.Show(extract, view1)

ResetCamera(view1, renderer)

## Handle data with ParaView

In [9]:
# set scalar coloring
simple.ColorBy(extractDisplay, ('CELLS', 'pressure'))

# rescale color and/or opacity maps used to include current data range
extractDisplay.RescaleTransferFunctionToDataRange(True, False)

# show color bar/color legend
extractDisplay.SetScalarBarVisibility(view1, True)

# get color transfer function/color map for 'pressure'
pressureLUT = simple.GetColorTransferFunction('pressure')
pressureLUT.RescaleTransferFunction(0.0, 1.0)
pressureLUT.ApplyPreset('Rainbow Desaturated', True)

# get opacity transfer function/opacity map for 'pressure'
pressurePWF = simple.GetOpacityTransferFunction('pressure')
pressurePWF.RescaleTransferFunction(0.0, 1.0)

# change representation type, for example wireframe or volume rendering
extractDisplay.SetRepresentationType('Wireframe');
# extractDisplay.SetRepresentationType('Volume');

### Setup interacts with ipywidgets

#### Set representation

In [10]:
dropdown = widgets.Dropdown(
    options=['Wireframe', 'Volume'],
#     value='Wireframe',
    description='Representation:',
    style={'description_width': '100pt'}
)

def change_representation(change):
    if change['new'] == 'Wireframe':
        extractDisplay.SetRepresentationType('Wireframe')
    elif change['new'] == 'Volume':
        extractDisplay.SetRepresentationType('Volume')
        
dropdown.observe(change_representation, 'value')

#### Pause/unpause

In [11]:
pause_button = widgets.Button(
    description='Pause',
    button_style='warning',
    tooltip='Pause catalyst simulation',
    icon='pause',
    disabled=False
)
unpause_button = widgets.Button(
    description='Unpause',
    button_style='success', 
    tooltip='Unpause catalyst simulation',
    icon='play',
    disabled=True
)

def pause(button):
    pause_button.disabled = True
    unpause_button.disabled = False
    catalyst.SetPauseSimulation(True)

def unpause(button):
    pause_button.disabled = False
    unpause_button.disabled = True
    catalyst.SetPauseSimulation(False)

pause_button.on_click(pause)
unpause_button.on_click(unpause)

buttons = widgets.HBox([unpause_button, pause_button])

#### Show timestep

In [12]:
from threading import Thread
from time import sleep

timestep = widgets.Label(
    value="timestep: 0"
)

def show_timestep(widget):
    while True:
        step = catalyst.GetTimeStep()
        widget.value = "timestep: {}".format(step)
        sleep(0.5)
    
thread = Thread(target=show_timestep, args=(timestep,))
thread.start()

## Display widgets and renderer

In [13]:
widget_layout = widgets.HBox(children=[dropdown, buttons, timestep],
                             layout={'margin': '4px', 'justify_content': 'space-between'})
renderer.layout.height = '100%'

In [14]:
widgets.VBox(children=[widget_layout, renderer], layout={'height': '800px'}) 